In [6]:
from tqdm.notebook import tqdm, trange
RR = RealField(5000)
Q.<x> = LaurentPolynomialRing(QQ)
sp_t_poly = 1/2*x**-1 + 1/2*x

In [7]:
TiGER256 = {}
TiGER256['p'] = 128
TiGER256['q'] = 256
TiGER256['k1'] = 128 
TiGER256['k2'] = 4
TiGER256['h_s'] = 198
TiGER256['h_r'] = 198
TiGER256['h_e'] = 32
TiGER256['n'] = 1024
TiGER256['f'] = 5
TiGER256['thres'] = 256/2

TiGER192 = {}
TiGER192['p'] = 64
TiGER192['q'] = 256
TiGER192['k1'] = 64 
TiGER192['k2'] = 4
TiGER192['h_s'] = 84
TiGER192['h_r'] = 84
TiGER192['h_e'] = 84
TiGER192['n'] = 1024
TiGER192['f'] = 5
TiGER192['thres'] = 256/2

TiGER128 = {}
TiGER128['p'] = 128
TiGER128['q'] = 256 
TiGER128['k1'] = 64 
TiGER128['k2'] = 64 
TiGER128['h_s'] = 160
TiGER128['h_r'] = 128
TiGER128['h_e'] = 32
TiGER128['n'] = 512
TiGER128['f'] = 3
TiGER128['thres'] = 256/2

In [8]:
def unif_error_poly_(p, q):
    
    err_max = int(q/p/2)
    err_list = [i for i in range(err_max+1)]
    poly = 0
    for i in err_list:
        if i == 0 :
            poly += 2
        elif i == err_max :
            poly += x**err_max + x**(-err_max)
        else :
            poly += 2*x**i + 2*x**(-i)
    
    return poly/poly(1)

def dist_of_squared_(poly):
    res = 0
    for i, expo in enumerate(poly.exponents()):
        res += poly.coefficients()[i]*x**(expo**2)
    
    return res

def cumul_binom_(p, n, k):
    res = 0
    for i in range(k+1):
        res += binomial(n, i)*(p**i)*((1-p)**(n-i))
        
    return res

def normalize_poly_(poly):
    return poly/(poly(1))



In [9]:
def TiGER_DFR(scheme):
    n = scheme['n']
    p = scheme['p']
    q = scheme['q']
    k1 = scheme['k1']
    k2 = scheme['k2']
    h_r = scheme['h_r']
    h_e = scheme['h_e']
    h_s = scheme['h_s']
    f = scheme['f']
    thres = scheme['thres']
    
    uA = unif_error_poly_(p, q)
    uA = dist_of_squared_(uA)
    uA_norm = uA**n
    
    uB_p = unif_error_poly_(k1, q)
    uB_p = dist_of_squared_(uB_p)
    uB_p_norm = uB_p**n
    
    uB_dp = unif_error_poly_(k2, q)
    eB_dp = sp_t_poly*h_e/n+(1-h_e/n)
    
    g = uB_dp*eB_dp
    
    var_sA = QQ(h_s/n)
    var_sB_p = QQ(h_r/n)
    
    pfail = 0
    
    for g_expo in tqdm(g.exponents()):
        for uA_norm_expo in tqdm(uA_norm.exponents(), leave=False):
            if(uA_norm[uA_norm_expo]<RR(2**-200)): continue
            else:
                for uB_p_norm_expo in uB_p_norm.exponents():
                    if(uB_p_norm[uB_p_norm_expo]<RR(2**-200)): continue
                    else :

                        std_SC = sqrt(RR(var_sA*(uB_p_norm_expo+h_e)+var_sB_p*uA_norm_expo))
                        
                        T = RealDistribution('gaussian', std_SC)

                        pbitfail = RR(1-T.cum_distribution_function(RR(thres-g_expo))+T.cum_distribution_function(RR(-thres-g_expo)))
                        
                        pfail +=RR((1-cumul_binom_(pbitfail, n, f)))*uA_norm[uA_norm_expo]*uB_p_norm[uB_p_norm_expo]*g[g_expo]
                        
                        
                        #pfail -= (1-scipy.stats.binom.cdf(correction_bit, n, pbitfail))*poly_uA[i]*poly_uBprime[j]
                        
                        
                        """
                        print("poly_uA[i]: ", poly_uA[i])
                        print("poly_uBprime[j]: ", poly_uBprime[j])
                        print("std_SC: ", std_SC)
                        print("right prob: ", RR(1-(T.cum_distribution_function(thres-g))))
                        print("left prob: ", T.cum_distribution_function(-thres-g))
                        print("pbitfail: ", type(pbitfail))
                        print("cumul_binom: ", cumul_binom)
                        print("")
                        """
                        
    return [RR(pfail).n(1000), RR(log(RR(pfail)),2).n(1000)]

In [10]:
TiGER_DFR(TiGER128)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/513 [00:00<?, ?it/s]

  0%|          | 0/513 [00:00<?, ?it/s]

  0%|          | 0/513 [00:00<?, ?it/s]

  0%|          | 0/513 [00:00<?, ?it/s]

  0%|          | 0/513 [00:00<?, ?it/s]

  0%|          | 0/513 [00:00<?, ?it/s]

  0%|          | 0/513 [00:00<?, ?it/s]

[8.84444270194531022696241013296063766943406097727643392057798775619453718400272567156325799244375817187773398203790669855722898829527911807096005808121643805850768588378667279811277934100224488855365604225410103087570803597074925606812535760278835995033612534735194453395228322009825356228337268003371e-38,
 -85.3184442152758712212565750953053130094566938372150399007577412636938322325150897642516307939604434999880868463689623455994270641094230386718675557616562691275014137375342790070496453411883339475730470862594635976879738178939921485947223426005846959695074580005149383803456654229338601224618859888155]

In [11]:
#TiGER_DFR(TiGER192)

In [12]:
#TiGER_DFR(TiGER256)